# Data splitting 
Is the process of dividing a dataset into separate subsets to train and evaluate a machine learning model. The common types of splits are:
- Training Set:	Used to train the model	60-80%
- Validation Set: Used to tune hyperparameters and prevent overfitting	10-20%
- Test Set:	Used for final model evaluation	10-20%

Important Considerations:
* Stratification: Used for classification problems to ensure the same class distribution across splits.
* Shuffle: Helps avoid any bias due to the dataset’s original order.
* Time Series Consideration: In this kind of models, data should not be randomly split but preserve chronological order to prevent future data from leaking into training.
* Cross valiadation: train and validate folds of the data.

### Clasical test and training dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load dataset (Titanic as an example)
df = pd.read_csv("titanic.csv")

# Define features (X) and target variable (y)
X = df.drop(columns=['Survived'])  # Independent variables
y = df['Survived']  # Target variable

# Perform train-test split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the datasets
print(f"Train Set: {X_train.shape}, Test Set: {X_test.shape}")


### Test, training and validation set for hyperparameter tunning

In [ ]:
# First, split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the training data further into training and validation sets (80-10-10 rule), 0.125 of 80% = 10% of total data, leaving 70% for training.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)

print(f"Train Set: {X_train.shape}, Validation Set: {X_val.shape}, Test Set: {X_test.shape}")


### Stratified Train-Test Split (For Imbalanced Classification)
If we have imbalanced classes, stratification ensures that the split maintains the original class distribution.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Check class distribution
print("Class Distribution in Training Set:\n", y_train.value_counts(normalize=True))
print("Class Distribution in Test Set:\n", y_test.value_counts(normalize=True))


### Time-Series Data Splitting, Sequential Splitting/ Cross-Validation 
In time-series cross-validation, we cannot randomly shuffle data like in standard k-fold CV because future values should not be used to predict past values. So this technique helps in hyperparameter tuning & model selection,it follows an expanding window approach where each fold adds more past data.


In [ ]:
# AAPL.csv dataset (Apple stock price data).

import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

# Load sample time-series dataset (AAPL stock data)
df = pd.read_csv("AAPL.csv", parse_dates=["Date"], index_col="Date")

# Ensure data is sorted by date (crucial for time-series)
df = df.sort_index()
#print(df)

# Selecting features (X) and target (y)
X = df[['Close', 'Volume']]  # Feature: Closing price & volume
y = df['Close'].shift(-1)     # Target: Next day's closing price (forecasting task)

# Drop last row with NaN due to shifting
#print(X.tail(),y.tail())
X, y = X.iloc[:-1], y.iloc[:-1]

# Initialize time-series cross-validation
tscv = TimeSeriesSplit(n_splits=5) #Splits the dataset into 5 consecutive train-test sets.

# Perform time-series split
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index] #Ensures that past data is always before future data.
    y_train, y_test = y.iloc[train_index], y.iloc[test_index] 

    print(f"Fold {fold+1}:")
    print(f"Train indices: {train_index[0]} to {train_index[-1]}")
    print(f"Test indices: {test_index[0]} to {test_index[-1]}\n")



Fold 1:
Train indices: 0 to 18
Test indices: 19 to 34

Fold 2:
Train indices: 0 to 34
Test indices: 35 to 50

Fold 3:
Train indices: 0 to 50
Test indices: 51 to 66

Fold 4:
Train indices: 0 to 66
Test indices: 67 to 82

Fold 5:
Train indices: 0 to 82
Test indices: 83 to 98



### K-Fold Cross-Validation (For Robust Model Evaluation)
When the dataset is small, and we need to use all data efficiently or we want a more reliable evaluation metric instead of a single train-test split.


In [ ]:
# Manual K-Fold Cross-Validation, More control :Custom metrics, feature selection, hyperparameter tuning

# Initialize K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42) #5 equal-sized folds, shuffles the data, and ensures reproducibility.

# Model initialization
model = RandomForestClassifier(n_estimators=100, random_state=42)

# The accuracy for each fold is calculated and stored in
fold_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Train the model
    model.fit(X_train_fold, y_train_fold)

    # Validate the model
    y_pred = model.predict(X_val_fold)
    accuracy = accuracy_score(y_val_fold, y_pred)
    
    # Store score
    fold_scores.append(accuracy)
    print(f"Fold {fold}: Accuracy = {accuracy:.4f}")

# Compute and display final average score
print(f"\nMean Accuracy across folds: {np.mean(fold_scores):.4f}") #the mean accuracy across all folds is computed to provide an estimate of model performance.

In [ ]:
#Automated cross-validation, enables multi-core processing and the entire process—splitting, training, predicting, and scoring is implicit

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Define model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Define K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Use cross_val_score for automated evaluation with parallelization
scoring = make_scorer(accuracy_score)  # Define scoring metric (customizable)

logging.info("Starting cross-validation...")
scores = cross_val_score(model, X_train, y_train, cv=5, n_jobs=-1) #n_jobs=-1: This enables parallelization during cross-validation.

# Display results
logging.info(f"Cross-validation scores: {scores}")
logging.info(f"Mean Accuracy: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

